In [ ]:
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
client = r'CAL00001 Raw ClientA-Run1-Client.csv'
lab = r'CAL00001 Raw ClientA-Run1-Lab.csv'

In [ ]:
df_client = pd.read_csv(client, encoding='mac_roman')
df_lab = pd.read_csv(lab, encoding='mac_roman')

In [ ]:
title_index = 0
Backgrounds_num = 0
Measurements_num = 0

for i in range(len(df_client)):
    # get where the data start
    if 'DATA' in df_client.iloc[i].tolist()[0]:
        title_index = i + 1
        break
        
    # get the Background number
    elif 'Backgrounds' in df_client.iloc[i].tolist()[0]:
        Backgrounds_num = int(df_client.iloc[i].tolist()[2])
        
    # get the measurement number
    elif 'Measurements' in df_client.iloc[i].tolist()[0]:
        Measurements_num = int(df_client.iloc[i].tolist()[2])

In [ ]:
# store the title
title = df_client.iloc[title_index].values.tolist()

In [ ]:
# dataframe of the before xray data
df_before = df_client[title_index + 1:title_index + 1 + Backgrounds_num]
df_before.columns = title
df_before.drop(['Comment'],axis=1,inplace=True)

# change the column type to numeric
df_before = df_before.apply(pd.to_numeric, errors='ignore')

In [ ]:
assert(len(df_before) == Backgrounds_num)

In [ ]:
# dataframe of the beam data with open xray
df_data = df_client[title_index + 1 + Backgrounds_num:-Backgrounds_num]
df_data.columns = title
df_data.drop(['Comment'],axis=1,inplace=True)

# check the number of measurement for each beam
df_size = df_data.groupby(['Filter']).size().to_frame('count')

# change the column type to numeric
df_data = df_data.apply(pd.to_numeric, errors='ignore')

# get the equipment which has more than Measurements_num measurements
duplicate_equip = list(df_size[df_size['count'] > Measurements_num].index)

df_noDuplicate = df_data[~df_data.Filter.isin(duplicate_equip)]
df_duplicate = df_data[df_data.Filter.isin(duplicate_equip)]
df_first = df_duplicate[:90]
df_last = df_duplicate[-90:]

In [ ]:
assert(len(df_first) + len(df_last) + len(df_noDuplicate) == len(df_data))

In [ ]:
# dataframe of the after xray data
df_after = df_client[-Backgrounds_num:]
df_after.columns = title
df_after.drop(['Comment'],axis=1,inplace=True)

# change the column type to numeric
df_after = df_after.apply(pd.to_numeric, errors='ignore')

In [ ]:
assert(len(df_after) == Backgrounds_num)

In [ ]:
# average the data
df_before_mean = df_before.groupby(['Filter']).agg({'mean'})
df_noDuplicate_mean = df_noDuplicate.groupby(['Filter']).agg({'mean'})
df_first_mean = df_first.groupby(['Filter']).agg({'mean'})
df_last_mean = df_last.groupby(['Filter']).agg({'mean'})
df_after_mean= df_after.groupby(['Filter']).agg({'mean'})

In [ ]:
# change the column type to numeric (other method)
# df_data['Current1(pA)'] = pd.to_numeric(df_data['Current1(pA)'])
# df_data['Current2(pA)'] = pd.to_numeric(df_data['Current2(pA)'])
# df_data['P(kPa)'] = pd.to_numeric(df_data['P(kPa)'])
# df_data['T(MC)'] = pd.to_numeric(df_data['T(MC)'])
# df_data['T(Air)'] = pd.to_numeric(df_data['T(Air)'])
# df_data['T(SC)'] = pd.to_numeric(df_data['T(SC)'])
# df_data['H(%)'] = pd.to_numeric(df_data['H(%)'])